In [1]:
!pip install elt
!pip install dash
!pip install dash-bootstrap-components


  Preparing metadata (setup.py) ... done
  Created wheel for elt: filename=elt-0.0-py3-none-any.whl size=1271 sha256=ccea024ff3022c23608ae8020f261a58c6e4813f9d66601e93e594f654aa150b
  Stored in directory: /root/.cache/pip/wheels/8c/10/2d/070c807299d2c19e9643d889a65bbc336451513c9b52519c7b
Successfully built elt
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 7.8 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.3/229.3 kB 7.9 MB/s eta 0:00:00


In [10]:
# Import required libraries
import pandas as pd
import sqlite3
import glob
import os

# ------------------------------------------------------------------------------

def main(folder_of_files):
    # Ensure folder exists
    if not os.path.exists(folder_of_files):
        raise Exception(f"\nFolder '{folder_of_files}' not found!")

    # Connect to SQLite database (stored in Colab session)
    conn = sqlite3.connect('/content/operation.db')

    # Find all .xls files in the directory
    files = glob.glob(os.path.join(folder_of_files, "*.xls"))

    if not files:
        raise Exception("\nNo datasets found in the 'sources' folder!")

    for file in files:
        try:
            # Read the Excel file
            xl = pd.ExcelFile(file)

            # Loop through each sheet in the Excel file
            for sheet in xl.sheet_names:
                table_name = sheet.replace(' ', '_').upper()  # Format table name
                df_tmp = xl.parse(sheet)  # Read sheet into DataFrame

                # Load data into SQLite
                df_tmp.to_sql(table_name, conn, if_exists='append', index=False)

        except Exception as e:
            print(f"Error processing {file}: {e}")

    conn.commit()
    conn.close()
    print("Data successfully loaded into operation.db")

# ------------------------------------------------------------------------------
if __name__ == '__main__':
    try:
        # Ensure 'sources' folder is correctly referenced
        main('/content/sources')
    except sqlite3.Error as e:
        print(f"SQLite Error: {e}")
    except Exception as e:
        print(f"Error: {e}")


Data successfully loaded into operation.db


In [11]:
import sqlite3
import pandas as pd
from  sqlite3 import Error

# ------------------------------------------------------------------------------
def create_connection(dw_file):
    '''
    create connection with data warehouse
    '''
    try:
        conn = sqlite3.connect(dw_file)
        return conn

    except Error as e:
        print(e)

# ------------------------------------------------------------------------------
def create_table_sql(conn, sql_query):
    '''
    create denormalized table from sql_query
    '''
    try:
        cursor = conn.cursor()
        cursor.execute(sql_query)

    except Error as e:
        print(e)

# ------------------------------------------------------------------------------
def main():
    '''
    create denormolized framework for data
    '''
    database = 'data_warehouse.db'

    data_warehouse = '''

    CREATE TABLE IF NOT EXISTS MASTER_FILE(
        Year INT NOT NULL CHECK(length(Year) = 4),
        Quarter INT NOT NULL CHECK(Quarter <= 4 AND Quarter >= 1),
        CustomerName CHAR(30) NOT NULL,
        City CHAR(30) NOT NULL,
        Country CHAR(30) NOT NULL,
        Sales DECIMAL(10,2) NOT NULL,
        Discount DECIMAL(10,2) NOT NULL,
        Cost DECIMAL(10,2) NOT NULL,
        Profit DECIMAL(15,2) NOT NULL CHECK(Profit > 0),
        ProfitType CHAR(6) NOT NULL,
        Quantity INT NOT NULL,
        Products CHAR(30) NOT NULL,
        ProductCategory CHAR(30) NOT NULL,
        EmployeeName CHAR(30) NOT NULL,
        Latitude FLOAT NOT NULL,
        Longitude FLOAT NOT NULL
    );
    '''

    # create a database connection
    conn = create_connection(database)

    if conn is not None:

        # create
        create_table_sql(conn, data_warehouse)

        conn.commit()

# ------------------------------------------------------------------------------
if __name__ == '__main__':

    try:
        main()
    except:
        print('Database exists!')

In [12]:
'''
Setting up tables for operation database
'''
import sqlite3
from sqlite3 import Error

# ------------------------------------------------------------------------------
def create_connection(db_file):
    '''
    create a database connection to sqlite database
    '''
    try:
        conn = sqlite3.connect(db_file)
        return conn

    except Error as e:
        print(e)

    return conn

# ------------------------------------------------------------------------------
def create_table(conn, create_table_sql):
    '''
    create a table from the create_table_sql statement
    '''
    try:
        cursor = conn.cursor()
        cursor.execute(create_table_sql)

    except Error as e:
        print(e)

# ------------------------------------------------------------------------------
def main():
    '''
    create all tables
    '''
    database = 'operation.db'

    products = '''

    CREATE TABLE IF NOT EXISTS PRODUCTS(
        ProductID INTEGER NOT NULL UNIQUE PRIMARY KEY,
        ProductName CHAR(50),
        CategoryID INTEGER NOT NULL,
        SupplierID INTEGER NOT NULL
    );
    '''

    categories = '''

    CREATE TABLE IF NOT EXISTS CATEGORIES(
        CategoryID INTEGER NOT NULL UNIQUE PRIMARY KEY,
        CategoryName CHAR(50),
        Description CHAR(50)
    );
    '''

    suppliers = '''

    CREATE TABLE IF NOT EXISTS SUPPLIERS(
        SupplierID INTEGER NOT NULL UNIQUE PRIMARY KEY,
        Supplier CHAR(50),
        SupplierContact CHAR(50),
        SupplierCountry CHAR(50)
    );
    '''

    order_header = '''

    CREATE TABLE IF NOT EXISTS ORDER_HEADER(
        OrderID INTEGER NOT NULL UNIQUE PRIMARY KEY,
        OrderDate DATETIME,
        CustomerID INTEGER,
        EmployeeID INTEGER,
        FOREIGN KEY(CustomerID) REFERENCES CUSTOMERS(CustomerNumber),
        FOREIGN KEY(EmployeeID) REFERENCES EMPLOYEES(EmployeeID)
    );
    '''

    employees = '''

    CREATE TABLE IF NOT EXISTS EMPLOYEES(
        EmployeeID INTEGER NOT NULL UNIQUE PRIMARY KEY,
        EmployeeName CHAR(30) NOT NULL
    );
    '''

    customers = '''

    CREATE TABLE IF NOT EXISTS CUSTOMERS(
        CustomerNumber INTEGER NOT NULL UNIQUE PRIMARY KEY,
        ContactName CHAR(30),
        City CHAR(25),
        Country CHAR(50),
        Fax CHAR(15),
        Phone CHAR(15),
        Latitude FLOAT,
        Longitude FLOAT,
        Region CHAR(15),
        EmployeeID INTEGER
    );
    '''

    order_details = '''

    CREATE TABLE IF NOT EXISTS ORDER_DETAILS(
        ProductID INTEGER NOT NULL,
        OrderID INTEGER NOT NULL,
        Quantity INTEGER,
        Sales DECIMAL (15, 2),
        Discount DECIMAL(8,2),
        Costs DECIMAL(8,2),
        Profit DECIMAL(8,2),
        FOREIGN KEY(ProductID) REFERENCES PRODUCTS(ProductID),
        FOREIGN KEY(OrderID) REFERENCES ORDER_HEADER(OrderID),
        PRIMARY KEY(ProductID, OrderID)
    );
    '''

    # create a database connection
    conn = create_connection(database)

    # create Tables

    if conn is not None:

        # create products table
        create_table(conn, products)

        # create categories table
        create_table(conn, categories)

        # create supplier table
        create_table(conn, suppliers)

         # create order_header table
        create_table(conn, order_header)

        # create employee table
        create_table(conn, employees)

        # create customers table
        create_table(conn, customers)

        # create order_details table
        create_table(conn, order_details)


        conn.commit()

    else:
        print('\nError cannot create database connection.')

# ------------------------------------------------------------------------------
if __name__ == '__main__':
    try:
        main()
    except:
        print('\nDatabase exists!')

In [13]:
'''
connect with operation database
create a denormalized dataframe for dataware house
'''

import pandas as pd
import sqlite3
from sqlalchemy import create_engine
from sqlite3 import Error

# ------------------------------------------------------------------------------
def create_connection(db_file):
    '''
    create connection with operation database
    '''
    try:
        conn = sqlite3.connect(db_file)
        return conn

    except Error as e:
        return print (e)

# ------------------------------------------------------------------------------
def create_cursor(conn, sql_query):
    '''
    hold the rows returned by sql query
    '''
    try:
        cursor = conn.cursor()

    except:
        print('\nPlease check database path!')

    try:
        return cursor.execute(sql_query)

    except UnboundLocalError as u:
        print(u)

    else:
        print('\nTable in database and query does not match!')

# ------------------------------------------------------------------------------
def get_datawarehouse_dataframe():
    '''
    create a dataframe for dataware house
    '''

    db_file = 'operation.db'

    order_details_query = '''
    SELECT
        strftime('%Y', OH.OrderDate) as Year,
        CASE
            WHEN strftime('%m', OH.OrderDate) BETWEEN '01' AND '03' THEN 1
            WHEN strftime('%m', OH.OrderDate) BETWEEN '04' AND '06' THEN 2
            WHEN strftime('%m', OH.OrderDate) BETWEEN '07' AND '09' THEN 3
            ELSE '4'
            END AS Quarter,
        ContactName as CustomerName,
        City,
        Country,
        round(OD.Sales, 2) as Sales,
        round(OD.Discount, 2) as Discount,
        round(OD.Costs, 2) as Cost,
        round(OD.Profit, 2) as Profit,
        Quantity,
        CASE
            WHEN OD.Profit < 75 THEN 'Small'
            WHEN OD.Profit >= 75 AND Profit < 300 THEN 'Medium'
            ELSE 'Large'
        END AS ProfitType,
        P.ProductName as Products,
        CAT.CategoryName as ProductCategory,
        EmployeeName,
        Latitude,
        Longitude
    FROM
        ORDER_DETAILS as OD
    INNER JOIN ORDER_HEADER as OH
        ON OH.OrderID = OD.OrderID
    INNER JOIN CUSTOMERS as C
        ON C.CustomerNumber = OH.CustomerID
    INNER JOIN PRODUCTS as P
        ON P.ProductID = OD.ProductID
    INNER JOIN CATEGORIES as CAT
        ON CAT.CategoryID = P.CategoryID
    INNER JOIN EMPLOYEES as EM
        ON EM.EmployeeID = OH.EmployeeID
    '''

    try:
        conn = create_connection(db_file)
    except:
        print('\nCannot create connection to database!')

    try:
        cursor = create_cursor(conn, order_details_query)
    except:
        print('\nTable cannot be found!')

    try:
        master_df = pd.DataFrame(cursor, columns=[i[0] for i in cursor.description])
    except:
        print('\nCannot convert query into dataframe!')

    return master_df

# ------------------------------------------------------------------------------
if __name__ == '__main__':

    get_datawarehouse_dataframe()

In [14]:
import elt
import glob
import os
import sqlite3
from sqlite3 import Error

# ------------------------------------------------------------------------------

# Defining the dataset source folder path
file_path = '/content/sources'  # Adjust for Colab

# ------------------------------------------------------------------------------

def get_master_data_file():
    '''
    Create a denormalized dataframe for data warehouse
    '''
    try:
        return get_datawarehouse_dataframe()
    except Error as e:
        print(f"Error fetching master data: {e}")
        return None

# ------------------------------------------------------------------------------

def check_operation_exists():
    '''
    Check if operation.db exists in the datasets folder
    '''
    db_path = '/content/operation.db'  # Adjust for Colab
    return os.path.exists(db_path)

# ------------------------------------------------------------------------------

def check_datawh_exists():
    '''
    Check if data_warehouse.db exists
    '''
    db_path = '/content/data_warehouse.db'  # Adjust for Colab
    return os.path.exists(db_path)

# ------------------------------------------------------------------------------

def main():
    checker_operation = check_operation_exists()
    checker_warehouse = check_datawh_exists()

    if not checker_operation:
        print("\nError: Operation database (operation.db) not found. Please run the required scripts first.")
        return

    print("\nOperation database exists. Proceeding to dataframe generation...")
    master_df = get_master_data_file()

    if master_df is None:
        print("\nError: Could not generate the master dataframe. Exiting process.")
        return
    print("\nDataframe for data warehouse generated successfully.")

    if not checker_warehouse:
        print("\nError: Data warehouse database (data_warehouse.db) not found. Please run the required scripts first.")
        return

    # Insert or update data warehouse
    try:
        conn = sqlite3.connect('/content/data_warehouse.db')  # Adjust for Colab
        master_df.to_sql('MASTER_FILE', conn, if_exists='append', index=False)
        print("\nData warehouse has been updated successfully!")
        conn.commit()
        conn.close()
    except Error as e:
        print(f"\nError updating data warehouse: {e}")

# ------------------------------------------------------------------------------

if __name__ == '__main__':
    main()



Operation database exists. Proceeding to dataframe generation...

Dataframe for data warehouse generated successfully.

Data warehouse has been updated successfully!


In [15]:
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
import dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go

import pandas as pd
from sqlite3 import Error
import sqlite3
#global master_df

# ################################################################################
'''
create connection with data warehouse
extract information of interest from data warehouse
'''

def create_connection():

    try:
        # create connection with data warehouse
        conn = sqlite3.connect('/content/data_warehouse.db')
        return conn

    except Error as e:
        print(e)

def fetch_master_df(conn):
    try:
        # create cursor to get data
        cursor = conn.cursor()

        # insert query into cursor
        cursor.execute('''
        SELECT * FROM MASTER_FILE
        ''')

        # convert query into dataframe
        return pd.DataFrame(cursor.fetchall(), columns=[i[0] for i in cursor.description])

    except Error as e:
        print(e)

# ################################################################################
# master file for satisfy visualization requirements

# create connection
conn = create_connection()

# get full data frame
master_df = fetch_master_df(conn)

# Check if master_df is None and handle the error
if master_df is None:
    print("Error: Could not fetch data from the database. Please check the database connection and data.")
    # You can choose to exit the program here or handle the error in a different way.
    # For example, you can display an error message on the dashboard or use a default dataset.
    # exit(1) # Example of exiting the program

# ------------------------------------------------------------------------------
# KPI and measures
def get_revenue(master_df):

    try:
        return '$ {0:,.2f}K'.format((master_df['Sales'].sum()/1000))

    except Error as e:
        print(e)
        pass

def get_expenses(master_df):

    try:
        return '$ {0:,.2f}K'.format((master_df['Cost'].sum()/1000))

    except Error as e:
        print(e)
        pass

def get_sales_quantity(master_df):
    try:
        return '{0:} Units'.format((master_df['Quantity'].sum()))

    except Error as e:
        print(e)
        pass

def get_discount(master_df):
    try:
        return '$ {0:,.2f}K'.format((master_df['Discount'].sum()/1000))

    except Error as e:
        print(e)
        pass

def get_profit(master_df):
    try:
        return '$ {0:,.2f}K'.format((master_df['Profit'].sum()/1000))

    except Error as e:
        print(e)
        pass

def get_sales_margin(master_df):
    try:
        total_profit = master_df['Profit'].sum()
        total_sales = master_df['Sales'].sum()
        # Check if total_sales is 0 to avoid ZeroDivisionError
        if total_sales == 0:
            return '0.00%'  # or any other appropriate value
        else:
            return '{:,.2f}%'.format((total_profit / total_sales) * 100)
    except Error as e:
        print(e)
        pass # or return a default value like 'N/A'

# ------------------------------------------------------------------------------
# Scatter plot of Sales vs Profit by Category
requirement_iii = master_df[['Sales', 'Discount', 'Cost', 'Profit', 'ProductCategory']].groupby('ProductCategory').sum().reset_index()

viz_scatter_iii = px.scatter(requirement_iii, x="Sales", y="Profit", size="Profit", color="ProductCategory",
           hover_name="Profit", log_x=True, log_y=True, size_max=50, template="seaborn", title='SALES AGAINST PROFIT BY PRODUCT CATEGORY')

layout = viz_scatter_iii.update_layout(
            hoverlabel=dict(
                bgcolor="white",
                font_size=16,
                font_family="Open Sans"
            ),
            height=700,
            width=700,
            title={
                'y':0.9,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
                font=dict(
                size=16,
                color="#4a4a4a",
                ),paper_bgcolor="#f8f9fa")

viz_scatter_iii.update_traces(mode="markers", hovertemplate='Sales: %{x} <br>Profit: %{y}')

# ------------------------------------------------------------------------------
# Sales by City
# get sales from master file, aggregate on country
city_sales = master_df[['Sales','Profit','Cost','Quantity','Discount','City']].groupby('City').sum()
lat_log_city = master_df[['City', 'Latitude', 'Longitude']].drop_duplicates().set_index('City')
requirement_ii = pd.concat([city_sales, lat_log_city], axis=1, join='inner').reset_index()
requirement_ii['Field'] = ' City' +'<br>' + requirement_ii['City'] + '<br>Sales ' + '$' + (requirement_ii['Sales'].round(2).astype(str))

mapbox_access_token = 'pk.eyJ1IjoiY2hha3JpdHRob25nZWsiLCJhIjoiY2tkdTAzd2hwMDBkZzJycWluMnFicXFhdCJ9.JjJhMoek5126u1B_kwYNiA'

px.set_mapbox_access_token(mapbox_access_token)

map_data = px.scatter_mapbox(requirement_ii, lat="Latitude", lon="Longitude", color="Sales", size="Sales",
                  color_continuous_scale=px.colors.cyclical.Edge, size_max=20, zoom=1,
                        center=dict(lon=-40.200033, lat=32.249974),
                        hover_data={'City':False, 'Latitude':False, 'Longitude':False, 'Sales':False, 'Field':True}, title='SALES BY CITY', template="seaborn")

map_data.update_layout(
                hoverlabel=dict(
                    bgcolor="white",
                    font_size=16,
                    font_family="Open Sans"
                ),
                height=600,
                width=700,
                title={
                    'y':0.9,
                    'x':0.5,
                    'xanchor': 'center',
                    'yanchor': 'top'},
                font=dict(
                size=16,
                color="#4a4a4a"),
                paper_bgcolor="#f8f9fa"
                    )

# --------------------------------------------------------------------------------
# Dash Table
data_table = master_df[['Year', 'Quarter', 'Country', 'City', 'CustomerName', 'ProfitType', 'ProductCategory', 'Sales', 'Profit', 'Cost']]

# ################################################################################
# initilize dash
# Bootstrap Javascript.

BS = "https://stackpath.bootstrapcdn.com/bootswatch/4.5.2/litera/bootstrap.min.css"

app = dash.Dash(__name__, external_stylesheets=[BS])
server = app.server

app.title = 'Mock Company Dashboard'
# Define app layout

app.layout = html.Div(
    html.Div([
        html.Div(
            [
                 html.H1(
                'Sales Department Data Analysis',
                style={
                    'fontSize':40,
                    'textAlign':'center',
                    'textDirection': 'vertical',
                    'dir':'rtl',
                    'padding': '20px',
                    'padding-top': '70px',
                    'color': '#41444b',
                    'margin-left':'auto',
                    'margin-right':'auto'
                    },
                        className='eight columns'),

                html.Img(
                    src="/assets/images/logo.png",
                    className='four columns',
                    style={
                        'height': '7%',
                        'width': '7%',
                        'float': 'left',
                        'position': 'relative',
                        'margin-top': 10,
                        'align':'center'
                    },
                ),
            ], className="row"
        ),

        dbc.Row([

            dbc.Col(html.Div(
                dbc.Alert("QUANTITY " +
                    get_sales_quantity(master_df), color='light')), width=2),

                dbc.Col(html.Div(
                dbc.Alert("REVENUE " +
                    get_revenue(master_df), color='light',)), width=2),

                dbc.Col(html.Div(
                    dbc.Alert("EXPENSES " +
                        get_expenses(master_df), color='light')), width=2),

                dbc.Col(html.Div(
                    dbc.Alert("DISCOUNT " +
                        get_discount(master_df), color='light')), width=2),

                dbc.Col(html.Div(
                    dbc.Alert("PROFIT " +
                        get_profit(master_df), color='light')), width=2),

                dbc.Col(html.Div(
                    dbc.Alert("MARGIN " +
                        get_sales_margin(master_df), color='light')), width=2),

        ], align="center",
            justify="center"),

        html.Div(
            [
            html.Div([
                dcc.Dropdown(
                            id = 'Cities',
                            options=[
                                {'label': 'Country', 'value': 'Country'},
                                {'label': 'Employee', 'value': 'EmployeeName'}
                            ],
                            value='Country',
                            style={'height': 'auto', 'width': '700px', 'align':'center'}
                        ),
                dcc.Graph(
                    id='bar_graph'
                )
                ], className= 'six columns',
                    style={'border-radius': '15px',
                            'backgroundColor': '#f8f9fa',
                            'box-shadow' : '4px 4px 2.5px #dddddd',
                            'padding':'20px', 'margin-left':'auto','margin-right':'auto', 'margin-top':'25px'}
                ),

            html.Div([
                dcc.Graph(
                    id='graph-4',
                    figure=map_data
                )
                ], className= 'six columns',
                    style={'border-radius': '15px',
                            'backgroundColor': '#f8f9fa',
                            'box-shadow' : '4px 4px 2.5px #dddddd',
                            'padding':'20px', 'margin-left':'auto','margin-right':'auto', 'margin-top':'25px'}
                )
            ], className="row"
        ),


        html.Div(
            [
            html.Div([
                dash_table.DataTable(
                        id='datatable-interactivity',
                        data=data_table.to_dict('records'),
                        columns=[{'name': i, 'id': i, "deletable":True,"selectable":True} for i in data_table.columns],
                        editable = True,
                        sort_action="native",
                        sort_mode="multi",
                        column_selectable="single",
                        row_selectable="multi",
                        row_deletable=True,
                        selected_columns=[],
                        selected_rows=[],
                        page_action="native",
                        page_current= 0,
                        page_size=55,
                        style_cell={
                        'textOverflow': 'ellipsis',
                        'overflow': 'hidden'
                        },
                        style_table={
                        'width':'700px',
                        'height':'600px',
                        'overflowY':'auto',
                        'overflowX': 'auto',
                        'align': 'center'
                        },
                        style_header={
                        'backgroundColor': 'rgb(230, 230, 230)',
                        'fontWeight': 'bold'
                        }
                )
                ], className= 'six columns',
                    style={'border-radius': '15px',
                                'backgroundColor': '#f8f9fa',
                                'box-shadow' : '4px 4px 2.5px #dddddd',
                                'padding':'20px', 'margin-left':'auto','margin-right':'auto', 'margin-top':'25px'}),
            html.Div([
                dcc.Graph(
                    id='graph-1', figure=viz_scatter_iii
                )
                ],  className= 'six columns',
                    style={'border-radius': '15px',
                            'backgroundColor': '#f8f9fa',
                            'box-shadow' : '4px 4px 2.5px #dddddd',
                            'padding':'20px', 'margin-left':'auto','margin-right':'auto', 'margin-top':'25px'}
                ),
            ], className="row"
        )
    ], className='ten columns offset-by-one')
)

# ################################################################################
# Connect the Plotly graphs with Dash Components

# Interactivity with table

@app.callback(
    dash.dependencies.Output('bar_graph', 'figure'),
    [dash.dependencies.Input('Cities', 'value')])
def select_cat(selector):
    revenue_country = master_df[['Country','Sales', 'EmployeeName']]
    revenue_country = revenue_country.groupby(selector).sum().reset_index().sort_values('Sales')

    revenue_country_viz = px.bar(revenue_country, y=selector, x='Sales', text='Sales', template="seaborn",
            hover_data={'Sales':':.2f'}, title='REVENUE BY ' + str(selector).upper())

    revenue_country_viz.update_traces(texttemplate='%{text:.2s}', textposition='outside')
    revenue_country_viz.update_layout(uniformtext_minsize=7, uniformtext_mode='hide', height=600)

    revenue_country_viz.update_layout(
                hoverlabel=dict(
                    bgcolor="white",
                    font_size=16,
                    font_family="Open Sans"
                ),
                title={
                        'y':0.9,
                        'x':0.5,
                        'xanchor': 'center',
                        'yanchor': 'top'},
                        font=dict(
                        size=16,
                        color="#4a4a4a"
                        ),
                        paper_bgcolor="#f8f9fa")
    return revenue_country_viz
# def update_table(input_value):
#     return generate_table(df_table.sort_values([input_value], ascending=[False]).reset_index())

# @app.callback(
#     Output(component_id='bar-chart', component_property='figure'),
#     [Input(component_id='bubble-chart', component_property='hoverData')]
# )

# def update_graph(master_df):
#    return bar(master_df)


# ################################################################################
if __name__ == '__main__':

    app.run_server(debug=True)

<IPython.core.display.Javascript object>